# Run ABC

## Chr2
working on Atmosphere instance in `/vol_c/ABC_AJmodels_update`.
Working with file,
`input_ABC_OSG_CHTC_HPC_chr2.txt`
which is a combination of simulations of chr2 with updated prior built from chr1 posterior from HPC, OSG, and CHTC, and has 162,752 simulations.

Use PLS to estimate parameters.

In [3]:
import sys, os
import pandas as pd
os.chdir('/vol_c/ABC_AJmodels_update')
!ls

input_ABC_HPC_chr2.txt		 RMSE_input_ABC_OSG_CHTC_HPC_chr2.txt.pdf
input_ABC_OSG_CHTC_chr2.txt	 Routput_input_ABC_OSG_CHTC_HPC_chr2.txt
input_ABC_OSG_CHTC_HPC_chr2.txt


In [3]:
!wc -l input_ABC_OSG_CHTC_HPC_chr2.txt

162752 input_ABC_OSG_CHTC_HPC_chr2.txt


### 0. Remove rows with missing data

In [5]:
pd.read_csv('input_ABC_OSG_CHTC_HPC_chr2.txt', sep = '\t', low_memory=False).dropna(axis=0, how='any').to_csv('input_ABC_OSG_CHTC_HPC_chr2_dropna.txt', sep='\t', index=False)

In [6]:
%%bash
mv input_ABC_OSG_CHTC_HPC_chr2_dropna.txt input_ABC_OSG_CHTC_HPC_chr2.txt

In [7]:
!wc -l input_ABC_OSG_CHTC_HPC_chr2.txt

162752 input_ABC_OSG_CHTC_HPC_chr2.txt


After removing missing data, there are 162,752 simulations.

## 1. Find PLS components
Needed to install R package "pls"
```
R
>install.packages("pls")
```

Use rscript, provided by Consuleo (from old version of ABCtoolbox)
```
Rscript /vol_c/src/abctoolbox-public/findPLS.r /vol_c/ABC_AJmodels_update/ input_ABC_OSG_CHTC_HPC_chr2.txt 24 204 1 23 180
```

This creates the output files  
`Routput_input_ABC_OSG_CHTC_HPC_chr2.txt`
`RMSE_input_ABC_OSG_CHTC_HPC_chr2.pdf`

Took 25 min to run.

## 2. Transform stats

In [10]:
file = open("test_ABC_transform_real.txt","w") 
file.write("task transform\n") 
file.write("input /vol_c/ABC_data/chr2/real_output_M23_IBD.summary\n") 
file.write("output real_output_M23_IBD_chr2_transformed.txt\n")
file.write("linearComb Routput_input_ABC_OSG_CHTC_HPC_chr2.txt\n")
file.write("boxcox 1\n")
file.write("logFile real_output_M23_IBD_chr2_transformed.log\n")
file.close() 

In [11]:
%%bash
ABCtoolbox test_ABC_transform_real.txt


 ABCtoolbox 2.0 
****************
- Reading inputfile 'test_ABC_transform_real.txt' ... done!
- Running in silent mode (use 'verbose' to get a status report on screen)




In [21]:
file = open("test_ABC_transform_sim.txt","w") 
file.write("task transform\n") 
file.write("input input_ABC_OSG_CHTC_HPC_chr2.txt\n") 
file.write("output input_ABC_OSG_CHTC_HPC_chr2_transformed.txt\n")
file.write("linearComb Routput_input_ABC_OSG_CHTC_HPC_chr2.txt\n")
file.write("boxcox 1\n")
file.write("logFile input_ABC_OSG_CHTC_HPC_chr2_transformed.log\n")
file.close() 

In [13]:
%%bash
ABCtoolbox test_ABC_transform_sim.txt


 ABCtoolbox 2.0 
****************
- Reading inputfile 'test_ABC_transform_sim.txt' ... done!
- Running in silent mode (use 'verbose' to get a status report on screen)




## 3. Estimate parameters

### 3.a. Reduce number of components to use for estimation

In [14]:
%%bash
cut -f-24,25-34 input_ABC_OSG_CHTC_HPC_chr2_transformed.txt >input_ABC_OSG_CHTC_HPC_chr2_transformed_10pls.txt
cut -f2-11 real_output_M23_IBD_chr2_transformed.txt >real_output_M23_IBD_chr2_transformed_10pls.txt

In [15]:
%%bash
cut -f10-15,18,20-23 input_ABC_OSG_CHTC_HPC_chr2_transformed_10pls.txt | head -2

TMJ	TAEW	Tm	Asc_NAF	Asc_NEU	Asc_NCHB	Log10_NCEU	Tgrowth_Af	TAF	Teu_as	TAg
342.273	15.2525	20.7778	8	14	17	3.76938	1264	2557	2541	5


### 3.b. Estimate with ABCtoolbox

In [19]:
sim_number = sum(1 for line in open('input_ABC_OSG_CHTC_HPC_chr2_transformed_10pls.txt'))

file = open("test_ABC_estimate_PLS.txt","w") 
file.write("task estimate\n")
file.write("simName input_ABC_OSG_CHTC_HPC_chr2_transformed_10pls.txt\n")
file.write("obsName real_output_M23_IBD_chr2_transformed_10pls.txt\n")
file.write("params 2-12\n")
file.write("numRetained 1000\n")
file.write("maxReadSims {}\n".format(sim_number))
file.write("diracPeakWidth 0.01\n")
file.write("posteriorDensityPoints 100\n")
file.write("jointPosteriors Log10_NWA,Log10_NEA\n")
file.write("jointPosteriorDensityPoints 100\n")
file.write("writeRetained 0\n")
file.write("outputPrefix ABC_M2_genome_estimate_{}_10pls_1000ret_chr2_\n".format(sim_number))
file.write("logFile ABC_M2_genome_estimate_{}_10pls_1000ret_chr2.log\n".format(sim_number))
file.write("verbose\n")
file.close()

In [20]:
%%bash
ABCtoolbox test_ABC_estimate_PLS.txt


 ABCtoolbox 2.0 
****************
- Reading inputfile 'test_ABC_estimate_PLS.txt' ... done!
- Writing log to 'ABC_M2_genome_estimate_162752_10pls_1000ret_chr2.log'
- Initializing random generator ... done with seed 157773528!
- Reading observed data file 'real_output_M23_IBD_chr2_transformed_10pls.txt' ... done!
   -> 1 data sets with 10 statistics each.
- Reading files with simulations:
   - Reading up to 162752 simulations from file 'input_ABC_OSG_CHTC_HPC_chr2_transformed_10pls.txt' ... done!   
      -> 162751 simulations with 11 parameters and 10 statistics each.
   - Checking for highly correlated statsitics:
      - Calculate pairwise correlations ...done!
      - No correlations exceeded the chosen threshold of 0.95.
   - Checking for highly correlated parameters:
      - Calculate pairwise correlations ...done!
      - No correlations exceeded the chosen threshold of 0.95.
- Performing an ABC-GLM standard estimations:
   - Standardizing Parameters ... done!
   - Performing es

In [24]:
%%bash
ls ABC_M2_genome_estimate_*_10pls_1000ret_chr2_*

ABC_M2_genome_estimate_162752_10pls_1000ret_chr2_model0_BestSimsParamStats_Obs0.txt
ABC_M2_genome_estimate_162752_10pls_1000ret_chr2_model0_jointPosterior_1_2_Obs0.txt
ABC_M2_genome_estimate_162752_10pls_1000ret_chr2_model0_MarginalPosteriorCharacteristics.txt
ABC_M2_genome_estimate_162752_10pls_1000ret_chr2_model0_MarginalPosteriorDensities_Obs0.txt
ABC_M2_genome_estimate_162752_10pls_1000ret_chr2_modelFit.txt


In [25]:
pd.read_csv('ABC_M2_genome_estimate_162752_10pls_1000ret_chr2_model0_MarginalPosteriorCharacteristics.txt', sep = '\t')

,dataSet,Log10_NWA_mode,Log10_NWA_mean,Log10_NWA_median,Log10_NWA_q50_lower,Log10_NWA_q50_upper,Log10_NWA_q90_lower,Log10_NWA_q90_upper,Log10_NWA_q95_lower,Log10_NWA_q95_upper,...,Tm_q99_lower,Tm_q99_upper,Tm_HDI50_lower,Tm_HDI50_upper,Tm_HDI90_lower,Tm_HDI90_upper,Tm_HDI95_lower,Tm_HDI95_upper,Tm_HDI99_lower,Tm_HDI99_upper
0,0,3.82222,4.72574,4.69512,3.89899,5.52033,3.24845,6.3321,3.13773,6.48794,...,4.108,34.4626,15.1212,25.3696,8.9798,32.1398,7.28178,33.5455,4.89666,35


## Chr3
working on Atmosphere instance in `/vol_c/ABC_AJmodels_update`.
Working with file,
`input_ABC_OSG_CHTC_HPC_chr3.txt`
which is a combination of simulations of chr2 with updated prior built from chr1 posterior from HPC, OSG, and CHTC, and has 134,718 simulations.

Use PLS to estimate parameters.

In [8]:
import sys, os
import pandas as pd
os.chdir('/vol_c/ABC_AJmodels_update')
!ls *chr3*
chrom = 3

input_ABC_HPC_chr3.txt
input_ABC_OSG_CHTC_chr3.txt
input_ABC_OSG_CHTC_HPC_chr3_params.pdf
input_ABC_OSG_CHTC_HPC_chr3_pca.pdf
input_ABC_OSG_CHTC_HPC_chr3_stats.pdf
input_ABC_OSG_CHTC_HPC_chr3.txt
RMSE_input_ABC_OSG_CHTC_HPC_chr3.txt.pdf
Routput_input_ABC_OSG_CHTC_HPC_chr3.txt


In [2]:
!wc -l input_ABC_OSG_CHTC_HPC_chr3.txt

134718 input_ABC_OSG_CHTC_HPC_chr3.txt


### Varify simulations
- plot the distribution of the parameters, 
- plot distribution of summary statistics, 
- PCA of summary statsitics

```
Rscript /vol_c/src/macsswig_simsaj/dist_plot_stats.R /vol_c/ABC_AJmodels_update/input_ABC_OSG_CHTC_HPC_chr3.txt /vol_c/ABC_data/chr3/real_output_M23_IBD.summary /vol_c/ABC_AJmodels_update/header_M2.txt
```

In [4]:
!ls *chr3*.pdf

input_ABC_OSG_CHTC_HPC_chr3_params.pdf	input_ABC_OSG_CHTC_HPC_chr3_stats.pdf
input_ABC_OSG_CHTC_HPC_chr3_pca.pdf


### 0. Remove rows with missing data

In [5]:
pd.read_csv('input_ABC_OSG_CHTC_HPC_chr3.txt', sep = '\t', low_memory=False).dropna(axis=0, how='any').to_csv('input_ABC_OSG_CHTC_HPC_chr3_dropna.txt', sep='\t', index=False)

In [6]:
%%bash
mv input_ABC_OSG_CHTC_HPC_chr3_dropna.txt input_ABC_OSG_CHTC_HPC_chr3.txt

In [7]:
!wc -l input_ABC_OSG_CHTC_HPC_chr3.txt

134718 input_ABC_OSG_CHTC_HPC_chr3.txt


After removing missing data, there are 134,718 simulations.

## 1. Find PLS components
Needed to install R package "pls"
```
R
>install.packages("pls")
```

Use rscript, provided by Consuleo (from old version of ABCtoolbox)
```
Rscript /vol_c/src/abctoolbox-public/findPLS.r /vol_c/ABC_AJmodels_update/ input_ABC_OSG_CHTC_HPC_chr3.txt 24 204 1 23 180
```

This creates the output files  
`Routput_input_ABC_OSG_CHTC_HPC_chr3.txt`
`RMSE_input_ABC_OSG_CHTC_HPC_chr3.pdf`

Took 11 min to run.

## 2. Transform stats

In [9]:
file = open("test_ABC_transform_real.txt","w") 
file.write("task transform\n") 
file.write("input /vol_c/ABC_data/chr{}/real_output_M23_IBD.summary\n".format(chrom)) 
file.write("output real_output_M23_IBD_chr{}_transformed.txt\n".format(chrom))
file.write("linearComb Routput_input_ABC_OSG_CHTC_HPC_chr{}.txt\n".format(chrom))
file.write("boxcox 1\n")
file.write("logFile real_output_M23_IBD_chr{}_transformed.log\n".format(chrom))
file.close() 

In [10]:
%%bash
ABCtoolbox test_ABC_transform_real.txt


 ABCtoolbox 2.0 
****************
- Reading inputfile 'test_ABC_transform_real.txt' ... done!
- Running in silent mode (use 'verbose' to get a status report on screen)




In [11]:
file = open("test_ABC_transform_sim.txt","w") 
file.write("task transform\n") 
file.write("input input_ABC_OSG_CHTC_HPC_chr{}.txt\n".format(chrom)) 
file.write("output input_ABC_OSG_CHTC_HPC_chr{}_transformed.txt\n".format(chrom))
file.write("linearComb Routput_input_ABC_OSG_CHTC_HPC_chr{}.txt\n".format(chrom))
file.write("boxcox 1\n")
file.write("logFile input_ABC_OSG_CHTC_HPC_chr{}_transformed.log\n".format(chrom))
file.close() 

In [12]:
%%bash
ABCtoolbox test_ABC_transform_sim.txt


 ABCtoolbox 2.0 
****************
- Reading inputfile 'test_ABC_transform_sim.txt' ... done!
- Running in silent mode (use 'verbose' to get a status report on screen)




## 3. Estimate parameters

### 3.a. Reduce number of components to use for estimation

In [13]:
%%bash
cut -f-24,25-34 input_ABC_OSG_CHTC_HPC_chr3_transformed.txt >input_ABC_OSG_CHTC_HPC_chr3_transformed_10pls.txt
cut -f2-11 real_output_M23_IBD_chr3_transformed.txt >real_output_M23_IBD_chr3_transformed_10pls.txt

In [14]:
%%bash
cut -f10-15,18,20-23 input_ABC_OSG_CHTC_HPC_chr3_transformed_10pls.txt | head -2

TMJ	TAEW	Tm	Asc_NAF	Asc_NEU	Asc_NCHB	Log10_NCEU	Tgrowth_Af	TAF	Teu_as	TAg
64.2691	10.7273	22.3939	17	8	12	4.75815	2319	2143	1859	3


### 3.b. Estimate with ABCtoolbox

In [15]:
sim_number = sum(1 for line in open('input_ABC_OSG_CHTC_HPC_chr{}_transformed_10pls.txt'.format(chrom)))

file = open("test_ABC_estimate_PLS.txt","w") 
file.write("task estimate\n")
file.write("simName input_ABC_OSG_CHTC_HPC_chr{}_transformed_10pls.txt\n".format(chrom))
file.write("obsName real_output_M23_IBD_chr{}_transformed_10pls.txt\n".format(chrom))
file.write("params 2-12\n")
file.write("numRetained 1000\n")
file.write("maxReadSims {}\n".format(sim_number))
file.write("diracPeakWidth 0.01\n")
file.write("posteriorDensityPoints 100\n")
file.write("jointPosteriors Log10_NWA,Log10_NEA\n")
file.write("jointPosteriorDensityPoints 100\n")
file.write("writeRetained 0\n")
file.write("outputPrefix ABC_M2_genome_estimate_{}_10pls_1000ret_chr{}_\n".format(sim_number, chrom))
file.write("logFile ABC_M2_genome_estimate_{}_10pls_1000ret_chr{}.log\n".format(sim_number, chrom))
file.write("verbose\n")
file.close()

In [16]:
%%bash
ABCtoolbox test_ABC_estimate_PLS.txt


 ABCtoolbox 2.0 
****************
- Reading inputfile 'test_ABC_estimate_PLS.txt' ... done!
- Writing log to 'ABC_M2_genome_estimate_134718_10pls_1000ret_chr3.log'
- Initializing random generator ... done with seed 97571268!
- Reading observed data file 'real_output_M23_IBD_chr3_transformed_10pls.txt' ... done!
   -> 1 data sets with 10 statistics each.
- Reading files with simulations:
   - Reading up to 134718 simulations from file 'input_ABC_OSG_CHTC_HPC_chr3_transformed_10pls.txt' ... done!   
      -> 134717 simulations with 11 parameters and 10 statistics each.
   - Checking for highly correlated statsitics:
      - Calculate pairwise correlations ...done!
      - No correlations exceeded the chosen threshold of 0.95.
   - Checking for highly correlated parameters:
      - Calculate pairwise correlations ...done!
      - No correlations exceeded the chosen threshold of 0.95.
- Performing an ABC-GLM standard estimations:
   - Standardizing Parameters ... done!
   - Performing est

In [17]:
%%bash
ls ABC_M2_genome_estimate_*_10pls_1000ret_chr3_*

ABC_M2_genome_estimate_134718_10pls_1000ret_chr3_model0_BestSimsParamStats_Obs0.txt
ABC_M2_genome_estimate_134718_10pls_1000ret_chr3_model0_jointPosterior_1_2_Obs0.txt
ABC_M2_genome_estimate_134718_10pls_1000ret_chr3_model0_MarginalPosteriorCharacteristics.txt
ABC_M2_genome_estimate_134718_10pls_1000ret_chr3_model0_MarginalPosteriorDensities_Obs0.txt
ABC_M2_genome_estimate_134718_10pls_1000ret_chr3_modelFit.txt


In [18]:
pd.read_csv('ABC_M2_genome_estimate_134718_10pls_1000ret_chr3_model0_MarginalPosteriorCharacteristics.txt', sep = '\t')

,dataSet,Log10_NWA_mode,Log10_NWA_mean,Log10_NWA_median,Log10_NWA_q50_lower,Log10_NWA_q50_upper,Log10_NWA_q90_lower,Log10_NWA_q90_upper,Log10_NWA_q95_lower,Log10_NWA_q95_upper,...,Tm_q99_lower,Tm_q99_upper,Tm_HDI50_lower,Tm_HDI50_upper,Tm_HDI90_lower,Tm_HDI90_upper,Tm_HDI95_lower,Tm_HDI95_upper,Tm_HDI99_lower,Tm_HDI99_upper
0,0,3.74747,4.57085,4.48025,3.77956,5.32195,3.21805,6.19209,3.12036,6.38509,...,3.65082,34.4639,12.9184,23.8485,6.92057,31.2828,5.52227,32.899,4.04957,34.8384
